In [1]:
import os
import mokapot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
sys.path
import data_loader as dl

The purpose of this function is to clean up the original data so that we are not counting decoys or duplicate scans.

In [2]:
def filter_data(df, cutoff, prob_column='PEP'):
    #drop decoys
    df = df[df["decoy"]==False]
    #sort by qvalue
    df = df.sort_values(prob_column)
    #drop duplicate scans
    df = df.drop_duplicates(subset=["scan"], keep="first") #keep highest coring

    return df

This is how we get ride of decoys

In [3]:
def make_decoy_col_maxquant(row):
    if row["Proteins"].startswith("REV"):
        return False
    else:
        return True

Here we read in the data and clean it up before we run it thorugh MokaPot. 
*Need to not drop columns and instead choose which ones we will give to it instead. 
We also change the scan number name so that all of our files will match in the end so we can merge them into one megascript. 

In [4]:
#Reading in the data and formatting it for MokaPot
def get_data_for_MokaPot(filename):
    mq_df =  dl.clean_maxquant(filename)
    mq_df["target_column"] = mq_df.apply(make_decoy_col_maxquant, axis = 1)
    
    
    #Dropping any rows that are missing the sequence
    #mq_df['Sequence'].replace(' ', np.nan, inplace = True)
    #mq_df.dropna(subset=['Sequence'], inplace=True)
    
    mq_df = mq_df.rename(columns = {"scan": "ScanNr"})
    
    #dropping columns because they are not numerical entries, or they are missing values.
    #Change this to selecting what colums we are going to give it versus just dropping everything
    mq_df = mq_df.drop(columns = {
       'Identified', 'Sequence', 'Mass analyzer', 'Modifications','Modified sequence','Type','Fragmentation',
       'Proteins','Raw file', 'decoy', 'Elapsed time','Matched','Reverse','Mass','Precursor intensity',
        'Precursor apex fraction', 'Precursor apex offset time', 'Score', 'PEP', 'Reporter PIF', 
        'Reporter fraction', 'Intens Comp Factor', 'CTCD Comp'})
    
    #Keep: pre intensity, score
    #**Fix in parser so that anything without sequence is gone. 
    
    return mq_df

This gives us back a dataset that has had the decoys, duplicate scans, and rows without a sequence number dropped.It has not been ran through MokaPot. The purpose of this is to be able to count how many scans we originally have at or under a specific cutoff. 

In [5]:
def get_PreMokaPot_data(filename):
    mq_df =  dl.clean_maxquant(filename)
    
#Formatting and dropping any rows that are missing the sequence (Do we still want this??)
    mq_df['Sequence'].replace(' ', np.nan, inplace = True)
    mq_df.dropna(subset=['Sequence'], inplace=True)
    
    mq_df = filter_data(mq_df, 0.01)
    
    return  mq_df


This is code that was taken from the MokaPot program. It formats the data for the graphs. THe purpose of the graphs is to compare the preMokaPot data to the postMokaPot data and compare if we are getting more scans at or below a certain cutoff after running data through MokaPot. 
MaxQuant does not have a q value, so instead we are using the PEP. 

In [6]:
def plot_qvalues(df, level="psms", threshold=0.01, ax=None, **kwargs):
    qvals = df["PEP"]

    ax = plot_qvalues(qvals, threshold=threshold, ax=ax, **kwargs)
    ax.set_xlabel("q-value")
    ax.set_ylabel(f"Accepted {self._level_labs[level]}")

    return ax

In [7]:
def plot_qvalues(qvalues, threshold=0.01, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()

    # Calculate cumulative targets at each q-value
    qvals = pd.Series(qvalues, name="qvalue")
    qvals = qvals.sort_values(ascending=True).to_frame()
    qvals["target"] = 1
    qvals["num"] = qvals["target"].cumsum()
    qvals = qvals.groupby(["qvalue"]).max().reset_index()
    qvals = qvals[["qvalue", "num"]]

    zero = pd.DataFrame({"qvalue": qvals["qvalue"][0], "num": 0}, index=[-1])
    qvals = pd.concat([zero, qvals], sort=True).reset_index(drop=True)

    xmargin = threshold * 0.05
    ymax = qvals.num[qvals["qvalue"] <= (threshold + xmargin)].max()
    ymargin = ymax * 0.05

    # Set margins
    curr_ylims = ax.get_ylim()
    if curr_ylims[1] < ymax + ymargin:
        ax.set_ylim(0 - ymargin, ymax + ymargin)

    ax.set_xlim(0 - xmargin, threshold + xmargin)
    ax.set_xlabel("q-value")
    ax.set_ylabel(f"Discoveries")

    ax.step(qvals["qvalue"].values, qvals.num.values, where="post", **kwargs)

    return ax

In [8]:
#all the files we want to run through MokaPot here 
file_names = ["2ng_rep1", "2ng_rep2", "2ng_rep3", "2ng_rep4", "2ng_rep5", "2ng_rep6",
             "0.2ng_rep1", "0.2ng_rep2", "0.2ng_rep3", "0.2ng_rep4", "0.2ng_rep5", "0.2ng_rep6"]

We run each file through MokaPot and save the results

In [9]:
for file in file_names:
    mq_cleaned_df = get_PreMokaPot_data(file)
    mq_df = get_data_for_MokaPot(file)

    mq_for_MP = mokapot.dataset.LinearPsmDataset(mq_df, target_column = "target_column", spectrum_columns = "ScanNr", 
                                                 peptide_column = "peptide", protein_column=None, 
                                                 group_column=None, feature_columns=None, copy_data=True)
    results, models = mokapot.brew(mq_for_MP)

    results_df = results.psms
    results_df.to_csv("MokaPot_Output/MaxQuant/mq_" + file + ".csv")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/daishavanderwatt/Payne_Lab/SingleCellBenchMark/data/maxquant/msms02ng.txt.gz'

In [ ]:
mq_df


Here we graph the results for pre and post MokaPot. We are plotting MaxQuant's PEP values against the q values that MokaPot improved on. 

In [ ]:
plot_qvalues(mq_cleaned_df["PEP"], label="Pre-mokapot")
plt.title("Mokapot vs MaxQuant")
results.plot_qvalues(label="mokapot")
plt.legend(["preMoka", "Mokapot"])
plt.vlines(x = 0.01, ymin = 0, ymax = 12000)
plt.tight_layout()

plt.show()

In [ ]:
print("The number of PSMs found at or above 0.01: ") 
      
print("\t" + "MaxQuant: " + str(len(mq_cleaned_df[mq_cleaned_df['PEP'] <= 0.01])))

print("\t""MaxQuant and MokaPot: " + str(len(results.psms[results.psms['mokapot q-value'] <= 0.01])))

In [10]:
mq_df =  dl.clean_maxquant("2ng_rep1")
mq_df.columns


FileNotFoundError: [Errno 2] No such file or directory: '/Users/daishavanderwatt/Payne_Lab/SingleCellBenchMark/data/maxquant/msms02ng.txt.gz'